In [1]:
# Importa os módulos para a análise dos dados
%matplotlib
import numpy
import pandas
from pandas.core.frame import DataFrame

pandas.options.mode.chained_assignment = None

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [2]:
# CONSTANTES IMPORTANTES
SEMANA = 'Domingo Segunda Terça VILLA Quinta Sexta Sábado'.split(' ')
WEEK = {
    'Monday': 'Segunda', 
    'Tuesday': 'Terça', 
    'Wednesday': 'VILLA', 
    'Thursday': 'Quinta',
    'Friday': 'Sexta',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

# Derivei a fórmula 4n-2[n/3] pra explicar a progressão 1 é 4, 3 é 10. Tenho muito tempo livre.
# Como a função floor não tem inverso, tive que fazer uma tabela de valores
VALORES = [(4*n - 2*int(n/3)) for n in range(1, 20)] + [15]

BEBIDAS = {
    'litrao': (1000, .05), # mL, %EtOH
    'petra': (269, .048), # mL, %EtOH
    'therezopolis': (350, .05), # mL, %EtOH
}

# Períodos em que foi consumida Therezopolis em vez de Petra
THEREZOPOLIS = [
    (pandas.to_datetime('2024-06-07'), pandas.to_datetime('2024-08-09')),
    (pandas.to_datetime('2024-11-12'), pandas.to_datetime('2024-11-30'))
]

In [3]:
# FUNÇÕES AUXILIARES
# Converte o pix em mL de bebida consumida
def pix_to_mL(pix):
    # Verifica se é litrão
    if pix['Valor'] == 15:
        return 1000

    # Verifica se o pix ocorreu no período Therezopolis
    if THEREZOPOLIS[0][0] <= pix['Data Lançamento'] <= THEREZOPOLIS[0][1]:
        return 350 * (VALORES.index(pix['Valor']) + 1)
 
    return 269 * (VALORES.index(pix['Valor']) + 1)

# Converte a entrada em pix em mL de EtOH consumido com base no teor alcoólico
def pix_to_EtOH(pix):
    # Verifica se é litrão
    if pix['Valor'] == 15: return 1000 * .05

    # Verifica se o pix ocorreu no período Therezopolis
    if THEREZOPOLIS[0][0] <= pix['Data Lançamento'] <= THEREZOPOLIS[0][1]:
        return 350 * (VALORES.index(pix['Valor']) + 1) * .05

    return (269 * (VALORES.index(pix['Valor']) + 1)) * .048

In [4]:
# FUNÇÃO PRINCIPAL
# Inicializa o extrato, fazendo as devidas conversões e filtragens
def inicializar_entradas(csv: str, threshold: float) -> DataFrame:
    # Importa o extrato completo
    extrato = pandas.read_csv(csv)
    
    # Converte o valor dos pix pra float
    extrato['Valor'] = extrato['Valor'].str.replace(',', '')
    extrato['Valor'] = extrato['Valor'].astype(float)

    # Mantém apenas os pix de entrada e cujo valor esteja abaixo de 100
    entradas = extrato[extrato['Valor'] > 0]
    entradas = entradas[entradas['Valor'] < threshold]

    # Converte as datas para o tipo datetime e organiza em ordem ascendente
    entradas['Data Lançamento'] = pandas.to_datetime(entradas['Data Lançamento'], dayfirst=True).dt.normalize()
    entradas = entradas.sort_values(by='Data Lançamento', ascending=True)

    # Adiciona uma coluna para os dias da semana
    entradas.loc[:, 'Dia'] = entradas['Data Lançamento'].dt.day_name()
    entradas['Dia'] = entradas['Dia'].replace(WEEK)

    # Normaliza todos os nomes para uppercase
    entradas['Nome'] = entradas['Nome'].str.upper()

    # Filtra os pix que foram validados e os converte em dados de consumo
    entradas = entradas[entradas['Valor'].isin(VALORES)]
    entradas['Consumo (mL)'] = entradas.apply(pix_to_mL, axis=1)

    return entradas

In [5]:
# ESTATÍSTICAS INDIVIDUAIS
# Calcula o consumo de álcool/bebida para uma pessoa
def calcular_alcoolismo(extrato: DataFrame, nome: str):
    # Filtra o extrato para incluir apenas a pessoa e os valores válidos
    pessoa = extrato[extrato['Nome'] == nome]

    # Calculo e retorna o consumo total
    consumo = pessoa['Consumo (mL)'].sum()
    return consumo

# Gera um gráfico evidenciando a tendência de consumo semanal da pessoa especificada
def plotar_alcoolismo_semanal(extrato: DataFrame, nome: str):
    pessoa = extrato[extrato['Nome'] == nome]

    gasto_diario = pessoa.groupby('Dia')['Consumo (mL)'].sum().reindex(SEMANA)
    gasto_diario.plot(kind='bar', color='skyblue')

In [6]:
# ESTATÍSTICAS GLOBAIS
# Calcula o consumo global de álcool
def alcoolismo_global(extrato: DataFrame):
    # Agrupa e soma o consumo total de cada pessoa
    consumo = extrato.groupby('Nome')['Consumo (mL)'].sum()
    consumo = consumo.sort_values(ascending=False)

    return consumo